In [ ]:
# Implement Object Detection Using Transfer Learning on Food Dataset

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import models

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to fit ResNet input
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize using ImageNet statistics
])

# Load the datasets
train_dataset = datasets.ImageFolder(root='Dataset/food-4/train', transform=transform)
test_dataset = datasets.ImageFolder(root='Dataset/food-4/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Number of classes
num_classes = len(train_dataset.classes)
print("Classes:", train_dataset.classes)


Classes: ['pizza', 'risotto', 'steak', 'sushi']


In [3]:
# Load pre-trained ResNet model
resnet = models.resnet50(pretrained=True)  # You can choose resnet18, resnet34, etc.

# Freeze all layers except the final classifier
for param in resnet.parameters():
    param.requires_grad = False

# Modify the final layer
resnet.fc = nn.Linear(resnet.fc.in_features, num_classes)  # Change output to number of classes

# Move model to device
resnet = resnet.to(device)

C:\Program Files\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Program Files\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\HP/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|█████████████████████████████████████████████████████████████████████████████| 97.8M/97.8M [06:31<00:00, 262kB/s]


In [4]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)  # Only train the final layer

In [5]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move to device
            
            optimizer.zero_grad()  # Zero the gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize
            
            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

# Train the model
train_model(resnet, train_loader, criterion, optimizer, num_epochs=5)

Epoch 1/5, Loss: 0.6647
Epoch 2/5, Loss: 0.3769
Epoch 3/5, Loss: 0.3151
Epoch 4/5, Loss: 0.3139
Epoch 5/5, Loss: 0.2829


In [6]:
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    
    with torch.no_grad():  # Disable gradient computation for evaluation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move to device
            outputs = model(inputs)  # Forward pass
            _, predicted = torch.max(outputs.data, 1)  # Get predictions
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  # Count correct predictions
    
    print(f'Test Accuracy: {100 * correct / total:.2f}%')

# Evaluate the model
evaluate_model(resnet, test_loader)

Test Accuracy: 88.80%
